In [163]:
import pandas as pd
import numpy as np

df_mai = pd.read_csv('VF_mai_counties_Q222.csv')
df_indcom = pd.read_csv('VF_indcom_counties_Q222.csv')
df_census = pd.read_csv('census.csv')

In [164]:
def preprocess(df, value_name, split_str):
    df = df.melt(id_vars='cfips', var_name='month', value_name=value_name)
    df.month = df.month.apply(lambda x: x.split(split_str, 1)[1])
    df['month'] = pd.to_datetime(df['month'], format='%b%y').dt.to_period('M').apply(lambda x: x.to_timestamp())
    df['row_id'] = df.cfips.astype(str) + '_' + df['month'].dt.strftime('%Y-%m-%d')
    df = df.drop_duplicates()
    return df

In [165]:
df_indcom = df_indcom[~df_indcom.cfips.isna()]
df_indcom = df_indcom.drop_duplicates(subset='cfips')
df_indcom.cfips = df_indcom.cfips.astype(int)
df_base = pd.concat([df_indcom.iloc[:, 0:5], df_indcom.iloc[:, -1]], axis=1)
df_base = df_base.drop(columns='total_pop_20') # drop population 20, adding new data later
df_order = pd.concat([df_indcom.iloc[:, 0], df_indcom.iloc[:, 5:40]], axis=1)
df_order = preprocess(df_order, 'orders_rank', 'orders_rank_')
df_merch = pd.concat([df_indcom.iloc[:, 0], df_indcom.iloc[:, 40:75]], axis=1)
df_merch = preprocess(df_merch, 'merchants_rank', 'merchants_rank_')
df_gmv = pd.concat([df_indcom.iloc[:, 0], df_indcom.iloc[:, 75:110]], axis=1)
df_gmv = preprocess(df_gmv, 'gmv_rank', 'gmv_rank_')
df_avg_tr = pd.concat([df_indcom.iloc[:, 0], df_indcom.iloc[:, 145:180]], axis=1)
df_avg_tr = preprocess(df_avg_tr, 'avg_traffic', 'avg_traffic_')

In [166]:
df_merged = df_order
for i in [df_merch, df_gmv, df_avg_tr]:
    df_merged = pd.merge(df_merged, i, on=['cfips', 'month', 'row_id'], how='inner')

df_merged = pd.merge(df_merged, df_base, on='cfips')

In [167]:
df_encoded = pd.get_dummies(df_merged['groupflag'])
df_merged = df_merged.drop(columns='groupflag')
df_merged.month = pd.to_datetime(df_merged.month, format='%b%y')
df = pd.concat([df_merged, df_encoded], axis=1)

In [168]:
df = df.drop(columns=['state', 'county'])

def normalized_rank_change(df, column, n):
    col = df.groupby('cfips').apply(lambda x: (x[column] - x[column].shift()) / n).reset_index(drop=True)
    col = col.fillna(0)
    return col
n = df.cfips.nunique()

df['nrc_order'] = normalized_rank_change(df, 'orders_rank', n)
df['nrc_merch'] = normalized_rank_change(df, 'merchants_rank', n)
df['nrc_gmv'] = normalized_rank_change(df, 'gmv_rank', n)

In [169]:
tlt = pd.read_csv('10yr_treasury_rate.csv')

co_est = pd.read_csv('co-est2021-alldata.csv', encoding='latin-1')
co_est["cfips"] = co_est.STATE*1000 + co_est.COUNTY
co_est = co_est.drop(columns=['SUMLEV', "REGION", "DIVISION", "STATE", "COUNTY", "CTYNAME"], axis=1)
co_est = co_est.rename(columns={'STNAME': 'state'})

rent = pd.read_csv('county_rent_estimates.csv')

covid = pd.read_csv('covidStats.csv')

state_employment = pd.read_csv('state_employment.csv')

state_corp_tax = pd.read_csv('state_corporate_tax_rates.csv')

df_new = pd.merge(co_est, state_employment, on='state')
df_new = pd.merge(df_new, tlt, on='first_day_of_month')
df_new['month'] = pd.to_datetime(df_new.first_day_of_month)
df_new['year'] = df_new.month.dt.year
df_new = df_new.drop(columns='first_day_of_month')
df_new = pd.merge(df_new, state_corp_tax, on=['state', 'year'])
df_new.columns

Index(['state', 'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021',
       'NPOPCHG2020', 'NPOPCHG2021', 'BIRTHS2020', 'BIRTHS2021', 'DEATHS2020',
       'DEATHS2021', 'NATURALCHG2020', 'NATURALCHG2021',
       'INTERNATIONALMIG2020', 'INTERNATIONALMIG2021', 'DOMESTICMIG2020',
       'DOMESTICMIG2021', 'NETMIG2020', 'NETMIG2021', 'RESIDUAL2020',
       'RESIDUAL2021', 'GQESTIMATESBASE2020', 'GQESTIMATES2020',
       'GQESTIMATES2021', 'RBIRTH2021', 'RDEATH2021', 'RNATURALCHG2021',
       'RINTERNATIONALMIG2021', 'RDOMESTICMIG2021', 'RNETMIG2021', 'cfips',
       'non_inst_pop', 'total_pop', 'pct_non_inst_pop', 'employed',
       'pct_employed', 'unemployed', 'pct_unemployed', 'DGS10_last', 'month',
       'year', 'tax_rate', 'tax_rate_diff'],
      dtype='object')

In [170]:
df = pd.merge(df, covid, on=['row_id', 'cfips']) # merge covid stats
df = pd.merge(df, df_new, on=['cfips', 'month'])
df

,cfips,month,orders_rank,row_id,merchants_rank,gmv_rank,avg_traffic,avg_lifespan_mths,30k - 225k pop,< 30k pop,...,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed,DGS10_last,year,tax_rate,tax_rate_diff
0,1001,2019-08-01,668.0,1001_2019-08-01,342.0,693.0,51.7,12.4,1,0,...,2270281,57.7,2204772,56.0,65509,2.9,2.02,2019,6.03,0.89
1,1001,2019-09-01,511.0,1001_2019-09-01,550.0,396.0,49.4,12.4,1,0,...,2271806,57.7,2206223,56.0,65583,2.9,1.50,2019,6.03,0.89
2,1001,2019-10-01,723.0,1001_2019-10-01,699.0,825.0,54.8,12.4,1,0,...,2273771,57.7,2207294,56.0,66477,2.9,1.68,2019,6.03,0.89
3,1001,2019-11-01,624.0,1001_2019-11-01,491.0,571.0,47.1,12.4,1,0,...,2275680,57.7,2207568,56.0,68112,3.0,1.69,2019,6.03,0.89
4,1001,2019-12-01,1023.0,1001_2019-12-01,777.0,1026.0,52.7,12.4,1,0,...,2276674,57.7,2206498,55.9,70176,3.1,1.78,2019,6.03,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107970,56045,2022-02-01,NaN,56045_2022-02-01,NaN,NaN,203.4,17.5,0,1,...,290102,63.7,279063,61.3,11039,3.8,1.79,2022,0.00,0.00
107971,56045,2022-03-01,1198.0,56045_2022-03-01,1093.0,1351.0,193.4,17.5,0,1,...,290282,63.7,279691,61.3,10591,3.6,1.83,2022,0.00,0.00
107972,56045,2022-04-01,1308.0,56045_2022-04-01,1120.0,1306.0,211.5,17.5,0,1,...,290379,63.6,280512,61.5,9867,3.4,2.32,2022,0.00,0.00
107973,56045,2022-05-01,1300.0,56045_2022-05-01,1080.0,988.0,210.6,17.5,0,1,...,290691,63.7,281100,61.6,9591,3.3,2.89,2022,0.00,0.00


In [171]:
df = df.drop(columns=['state', 'year', 'month'])
df

,cfips,orders_rank,row_id,merchants_rank,gmv_rank,avg_traffic,avg_lifespan_mths,30k - 225k pop,< 30k pop,> 225k pop,...,non_inst_pop,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed,DGS10_last,tax_rate,tax_rate_diff
0,1001,668.0,1001_2019-08-01,342.0,693.0,51.7,12.4,1,0,0,...,3935925,2270281,57.7,2204772,56.0,65509,2.9,2.02,6.03,0.89
1,1001,511.0,1001_2019-09-01,550.0,396.0,49.4,12.4,1,0,0,...,3938622,2271806,57.7,2206223,56.0,65583,2.9,1.50,6.03,0.89
2,1001,723.0,1001_2019-10-01,699.0,825.0,54.8,12.4,1,0,0,...,3941513,2273771,57.7,2207294,56.0,66477,2.9,1.68,6.03,0.89
3,1001,624.0,1001_2019-11-01,491.0,571.0,47.1,12.4,1,0,0,...,3944209,2275680,57.7,2207568,56.0,68112,3.0,1.69,6.03,0.89
4,1001,1023.0,1001_2019-12-01,777.0,1026.0,52.7,12.4,1,0,0,...,3946542,2276674,57.7,2206498,55.9,70176,3.1,1.78,6.03,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107970,56045,NaN,56045_2022-02-01,NaN,NaN,203.4,17.5,0,1,0,...,455580,290102,63.7,279063,61.3,11039,3.8,1.79,0.00,0.00
107971,56045,1198.0,56045_2022-03-01,1093.0,1351.0,193.4,17.5,0,1,0,...,455939,290282,63.7,279691,61.3,10591,3.6,1.83,0.00,0.00
107972,56045,1308.0,56045_2022-04-01,1120.0,1306.0,211.5,17.5,0,1,0,...,456266,290379,63.6,280512,61.5,9867,3.4,2.32,0.00,0.00
107973,56045,1300.0,56045_2022-05-01,1080.0,988.0,210.6,17.5,0,1,0,...,456588,290691,63.7,281100,61.6,9591,3.3,2.89,0.00,0.00


In [172]:
df.columns

Index(['cfips', 'orders_rank', 'row_id', 'merchants_rank', 'gmv_rank',
       'avg_traffic', 'avg_lifespan_mths', '30k - 225k pop', '< 30k pop',
       '> 225k pop', 'nrc_order', 'nrc_merch', 'nrc_gmv', 'confirmed',
       'deaths', 'people_vaccinated', 'people_fully_vaccinated',
       'school_closing', 'workplace_closing', 'cancel_events',
       'gatherings_restrictions', 'transport_closing',
       'stay_home_restrictions', 'internal_movement_restrictions',
       'international_movement_restrictions', 'information_campaigns',
       'testing_policy', 'contact_tracing', 'facial_coverings',
       'vaccination_policy', 'elderly_people_protection',
       'government_response_index', 'stringency_index',
       'containment_health_index', 'economic_support_index',
       'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021',
       'NPOPCHG2020', 'NPOPCHG2021', 'BIRTHS2020', 'BIRTHS2021', 'DEATHS2020',
       'DEATHS2021', 'NATURALCHG2020', 'NATURALCHG2021',
       'INTERNATIONALM

In [173]:
df.to_csv('df_cleaned.csv')